# Clustering Conformations using Python Libraries


## Import dependencies

In [1]:
import numpy as np
import pickle
from scipy.cluster.hierarchy import dendrogram, linkage, leaves_list, to_tree, centroid, cut_tree,fcluster
from matplotlib import pyplot as plt
from helper import info, threshold_remove, multirun
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

<style type='text/css'>
.CodeMirror{
    font-size: 12px;
}

div.output_area pre {
    font-size: 12px;
}
</style>

### Isolate lists with missing residues beyond a threshold

#### threshold_remove(threshold, segments = None)
#### Parameters:
##### threshold, 
minimum number of missing residues before discarded, all conformations with missing residues ≥ threshold are discarded.
Segments, default None analyzes the complete sequence for missing residues.
Otherwise takes in a list of tuples (start, end) residue indices using zero indexing.

This method is for the purpose of threshold vs missclassification analysis. In this case the full length is kept since the whole chains_list will be analyzed regardless of the amount of missing indices.

In [2]:
threshold_remove(297)


Threshold: 297 amount removed: 0


## Load presorted classification based on annotations list

In [ ]:
#@ open files
with open("chains_list.var","rb") as chains_list_var:
   chains_list = pickle.load(chains_list_var)
   chains_list_var.close()
#print(f"chains_list: {chains_list}")
with open("structures/opened_active.var", "rb") as open_active_var:
    open_active_list = pickle.load(open_active_var)
    open_active_var.close()
with open("structures/closed_inactive.var", "rb") as closed_inactive_var:
    closed_inactive_list = pickle.load(closed_inactive_var)
    closed_inactive_var.close()
with open("structures/opened_inactive.var", "rb") as open_inactive_var:
    open_inactive_list = pickle.load(open_inactive_var)
    open_inactive_var.close()

print(f"open_active: {len(open_active_list)}")
print(f"closed_inactive: {len(closed_inactive_list)}")
print(f"open_inactive: {len(open_inactive_list)}")
annotated_dict_list_codes= {"open_active": open_active_list, "closed_inactive": closed_inactive_list, "open_inactive": open_inactive_list} #dictionary of codes list
annotated_dict_list ={"open_active": list(), "closed_inactive": list(), "open_inactive": list()} #dictionary of list of indices
for i,conformation in enumerate(chains_list):
    for j,l in enumerate(annotated_dict_list_codes):
        if conformation in annotated_dict_list_codes[l]:
            #print(f"l: {l}")
            annotated_dict_list[l].append(i)
#print(f"annotated_dict_list: {annotated_dict_list_codes}")


### Open calculated rms matrix and chains list
#### matrix.var
Calculated rms matrix from aligning all of the choice conformation chosen by best_align.py and calculated by pml_script_all.py.
#### chains_list.var
Generated by reading through the dictionary of annotated conformations and making a list of chains of the conformation. This will give the labeling order of the rms matrix obtained since pml_script_all.py iterated through this list

In [ ]:
with open("matrix.var", "rb") as matrix_var:
   matrix = (pickle.load(matrix_var))
   matrix_var.close()
#print(f"matrix: {matrix}")
with open("matrix_AB.var", "rb") as matrix_AB_var:
   matrix_AB = (pickle.load(matrix_AB_var))
   matrix_AB_var.close()
with open("matrix_seg.var", "rb") as matrix_seg_var:
   matrix_seg = pickle.load(matrix_seg_var)
   matrix_seg_var.close()

### Ward's Algorithm
A linkage with minimum variance method. We define 
$$
T = |v| + |s| + |t|
$$
As the cardinality the forest _u_ and _v_ combined
Given the distance function is computed recursively with the equation:
$$
d[i][j] = d(i,j) = \sqrt{\frac{|v| + |s|}{T}d(v,s)^2 + \frac{|v| + |t|}{T}d(v,t)^2 - \frac{|v|}{T}d(s,t)^2}
$$
The merging cost, of which ward's algorithm tries to minimize the growth is defined as follows:
$$
Cost(u,v) = \frac{|u||v|}{|u| +|v|}||c_u - c_v ||^2
$$
$c_i$ is the vector of the centroid. And the derivation comes from the distance sum union of the two sets together minus the distance sum of the individual sets. 
<br>
It considers the case of merging and then calculates the variance from the centroid of the final merged cluster as the distance

## Info function:
 ```info(matrix,title,chains_list,annotated_dict_list,complete=True,kernel="linear",hierarchy_method = None, no_clusters=3, dist_metric = "euclidean", tsne= False, auto_cut_tree = True)```
### Usage:
### __Required Parameters__:
#### matrix:
    A matrix of values where each row represents a conformation and the features along the columns
    In this case we have RMSD of all the CA, RMSD of all CA + CB, RSMD of all CA of specific segments
#### title: 
    Title of the matrix for plotting purposes
#### chains_list:
    List chains/ conformations in the same order as the matrix
#### annotated_dict_list:
    Dictionary where each key is group classified, found by pdb file annotation and value is a list of indices of the conformations on the matrix
#### complete:
    For the purposes of identifying the coincidence of misclassification and missing residues beyond a certain threshold. Complete is a boolean, if True it will use the full matrix of data. Else if false, that means chains_list is reduced_chains_list (a shorter chain where every conformation has less than threshold missing residues as defined by the last run threshold_reduce function)
    <b> if threshold_removed was called and removed any conformations and input chains_list is a reduced_chains_list then set this to true</b>
### __Optional Parameters__:
#### kernel:
    For the use of PCA, Kernel method of PCA (if in use)
#### Hierarchy_method:
    Method of hierachical clustering, default is None. set to None then the function will use the graph for Kmeans clustering by default. Uses sklearn and scipy hierarchy clustering. 
    Choices: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘cosine’, ‘precomputed’}
#### no_clusters:
    Number of clusters to be found. However, the statistics will give an error if the amount of clusters and the length of the ```annotated_dict_list``` do not match
#### dist_metric:
    Distance metric for hierarchical clustering, by default euclidean
#### tsne:
    Data is visualized in 2 different ways T-SNE and PCA, By default tsne = False means PCA is used. Data is projected into 2 dimensions
#### auto_cut_tree:
    By default the tree will be automatically cut by the number of clusters, however, if set to False, manual cutting will ask the user to get left or right tree in a sequence from top down.
    The first choice will prompt the user for the leftmost tree and then the second choice for the middle tree.
    Use the calculated dendrogram to decide on the manual tree partitioning.
    Use: "left", "l", "right", or "r"
    The statistics will give an erroneous result if the middle tree, the second tree is chosen not to the immediate right of the first tree.



## Experiment 1:
Complete CA, T-SNE projection with Ward's algorithm to obtain 3 clusters.

In [ ]:
info(matrix,"Complete RMSD, CA", chains_list,annotated_dict_list, kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=True)

### Experiment 3
Complete CA and CB RSMD Matrix, T-SNE projection with Ward's algorithm to obtain 3 clusters.
### Analysis:

In [ ]:
info(matrix_AB,"Complete RMSD CA and CB", chains_list,annotated_dict_list,kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=True)

### Experiment 5
Selected Segments 33 to 44, 150 to 159, found from previous pymol modelling analysis. RSMD Matrix, T-SNE projection with Ward's algorithm to obtain 3 clusters.

### Analysis:

In [ ]:
info(matrix_seg,"Selected RMSD complete (33:44) and (150:159) CA ", chains_list,annotated_dict_list,kernel="linear",hierarchy_method = "ward", no_clusters=3,tsne=True)

### Experiment 6 less intuitive partition
Selected Segments 33 to 44, 150 to 159, found from previous pymol modelling analysis. T-SNE projection with Ward's algorithm to obtain 3 clusters, Manual Tree parititioning, Cluster1 = Tree.left.left, Cluster2 = Tree.left.right, Cluster3 = Tree.right
### Analysis:
This gives quite a high F1 score, however the partitioning is not as intuitive since Cluster 1 and Cluster 2 are highly similar and have low cophenetic distances compared to the Cluster 3. Maybe a change of distance metric might give different clustering results.

Actually, running a script to check how many chains are missing more than 7 residues in on of the segments (keeping in mind that the segments are only 12 and 10 residues long). The script outputs 176 samples. This is more than the amount misclassified at 79 conformations.

Taking from the misclassified examples 2BHH_A on the top right of the graph of the misclassified conformation has a gap from residue 36 to 43 and 148 to 162 which given only two residues of course would have trouble identifying the clustering.
1AQ1_A missing from 36 to 44 and 148 to 160.

On the other side of the misclassified cluster, 2AOC_X has only missing 37-40, 4 key residues missing

In [ ]:
info(matrix_seg,"Selected RMSD complete (33:44) and (150:159) CA diffferent partition", chains_list,annotated_dict_list,hierarchy_method = "ward", no_clusters=3,tsne=True,auto_cut_tree=False)

## Multirun coordinate based on the full chain without threshold:
Now compare the results to coordinate based clustering method developped. Set the generic segment to be the full sequence. The threshold is 297 so none of the conformations will be thrown away

In [ ]:
multirun(annotated_dict_list,threshold=297,segments=None,iter=50)